In [ ]:
import os
import tensorflow        as tf
import numpy             as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import math
import random
import time
import json
from data_process        import *
from Model               import *
# from sklearn.manifold    import TSNE

NUM_SPEAKER    = 77
BATCH_START    = 0
BATCH_SIZE     = 30
TIME_STEP      = 20 
LR             = 0.001
lr = 0.01
lr_decay = 0.9
KEEP_PROB      = 1.0
EPOCH_UP       = 50
EPOCH_LOW      = 20
SPEAKERIDX     = 0
INPUT_SIZE     = 513
OUTPUT_SIZE    = 513
HIDDEN_SIZE    = 1000
K              = 2
TAO0           = 5.0
ANNEAL_RATE    = 0.1
MIN_TAU        = 0.5

ARCHITECTURE   = {'l0':{'type':'input', 'neurons':INPUT_SIZE}, 
                  'l1':{'type':'fc', 'neurons':1000}, 
                  'l2':{'type':'mrnn', 'neurons':HIDDEN_SIZE, 'state_size':K}, 
                  'l3':{'type':'fc', 'neurons':1000}, 
                  'l4':{'type':'output', 'neurons':OUTPUT_SIZE}}

# ARCHITECTURE   = {'l0':{'type':'input', 'neurons':INPUT_SIZE}, 
#                   'l1':{'type':'fc', 'neurons':1000}, 
#                   'l2':{'type':'lstm', 'neurons':HIDDEN_SIZE}, 
#                   'l3':{'type':'lstm', 'neurons':HIDDEN_SIZE}, 
#                   'l4':{'type':'fc', 'neurons':700}, 
#                   'l5':{'type':'output', 'neurons':OUTPUT_SIZE}}

TOTAL_SIZE     = 0

data           = 'data_merge'
SNR            = 5
URL            = '../'+data+'/SNR='+str(SNR)+'/'

# Training

In [ ]:
start = time.time()

mix_train, target1, target2, sequence_length_train = get_data_train(
    URL+'Train/Mix/', URL+'Train/Target1/', URL+'Train/Target2/', NUM_SPEAKER, BATCH_SIZE, TIME_STEP, longest=200)
end = time.time()
print('Get data done!', "    time: ", end-start)

In [ ]:
# start = time.time()
# np.save(URL+'Train/mix', mix_train)
# np.save(URL+'Train/t1', target1)
# np.save(URL+'Train/t2', target2)
# np.save(URL+'Train/sl', sequence_length)
# end = time.time()
# print(end-start)

In [ ]:
# start = time.time()
# mix = np.load(URL+'Train/mix.npy')
# t1 = np.load(URL+'Train/t1.npy')
# t2 = np.load(URL+'Train/t2.npy')
# sl = np.load(URL+'Train/sl')

# mix_train=mix_train.item()
# target1=target1.item()
# target2=target2.item()
# sequence_length=sl.item()

# end = time.time()
# print(end-start)

In [ ]:
tf.reset_default_graph()
model = Model(ARCHITECTURE, INPUT_SIZE, OUTPUT_SIZE, BATCH_SIZE, TIME_STEP, LR, activation_function=tf.nn.relu, batch_norm=True)
init  = tf.global_variables_initializer()
sess  = tf.Session()
sess.run(init)

In [ ]:
MSE = [np.inf] #np.zeros(EPOCH_UP)
total_cost = 0
model.assign_lr(sess, lr)
for ep in range(EPOCH_UP):
    # Training
    sp_list = [i for i in range(NUM_SPEAKER)]
    random.shuffle(sp_list)
    sp_idx = 0
    BATCH_idx = 0 # to record starting point of batch we want to extract
    TIME_idx  = 0
    start = time.time()
    np_temp=np.maximum(TAO0*np.exp(-ANNEAL_RATE*ep),MIN_TAU)
    model.assign_tau(sess, np_temp)
    temp_lr, temp_tau = sess.run([model.lr, model.tau])
    
    while True:
        mix, t1, t2, sequence, sp_idx, BATCH_idx, TIME_idx = get_batch_train(
            mix_train, target1, target2, BATCH_idx, TIME_idx, sp_idx, BATCH_SIZE, TIME_STEP, INPUT_SIZE, sp_list, 
            sequence_length_train, longest=200)     
        
        # break
        if sp_idx == NUM_SPEAKER:
            break

        feed_dict = {
                model.x  : mix,
                model.y1 : t1,
                model.y2 : t2,
                model.keep_prob : KEEP_PROB,
                model.training : True
            }
        
        model.sequence_length = sequence
        # stochastic gradient descent
        _, cost = sess.run(
            [model.train_op, model.cost],
            feed_dict
            )

        if math.isnan(cost):
            break
        model.init_state_assign()
        
        # cost
        total_cost = total_cost + cost
    end = time.time()  
    train_time = end-start
    
    TOTAL_SIZE = 0
    for key in sequence_length_train.keys():
        TOTAL_SIZE += sum(sequence_length_train[key])
        
    MSE.append(total_cost/(TOTAL_SIZE))
    total_cost = 0
    
    print 'Epoch: ', ep, ' Training Loss: ', MSE[-1], ' Training Time: ', train_time, ' Tau: ', temp_tau, ' LR: ', temp_lr
    
    model.assign_lr(sess, temp_lr * lr_decay)
    if MSE[-2] - MSE[-1] < 0 and ep > 20: #or ep_idx >= 3:
        break
    #if ep >= EPOCH_LOW and MSE[ep-1]-MSE[ep] < 0.001:
    #    MSE = MSE[0:ep+1]
    #    break;    
        
                        
print('----------End----------')

In [ ]:
total_para = 0
for var in tf.trainable_variables():
    shape = var.get_shape()
    tmp = 1
    for dim in shape:
        tmp *= dim.value
    total_para += tmp
print('Total parameters: ', total_para)
print(tf.trainable_variables())

# Plot

In [ ]:
%matplotlib inline
fig = plt.figure()
ax  = fig.add_subplot(1, 1, 1)

ax.plot(MSE)
plt.ion()
plt.show()

for idx in MSE:
    print idx

# Seen speaker test

In [ ]:
NUM_SPEAKER_TEST = 6
mix_test1, t1_test1, t2_test1, order, sequence_length = get_data_test(URL+'Test1/Mix/', URL+'Test1/Target1/', URL+'Test1/Target2/', 
                                                                      NUM_SPEAKER_TEST, BATCH_SIZE, TIME_STEP)

In [ ]:
total_cost  = 0
TOTAL_SIZE  = 0
total_pred1 = np.zeros((1, 513))
total_pred2 = np.zeros((1, 513))
sp_list     = mix_test1.keys()
sp_list.sort()
for speaker in range(NUM_SPEAKER_TEST):
    BATCH_START = 0 # to record starting point of batch we want to extract
    DATA_SIZE   = mix_test1[sp_list[speaker]].shape[0]
    TOTAL_SIZE  = TOTAL_SIZE + DATA_SIZE
    for idx in range((DATA_SIZE/(BATCH_SIZE*TIME_STEP))):
        BATCH_START = idx*BATCH_SIZE*TIME_STEP
        mix, t1, t2 = get_batch_test(
            mix_test1[sp_list[speaker]], t1_test1[sp_list[speaker]], t2_test1[sp_list[speaker]], 
            BATCH_START, BATCH_SIZE, TIME_STEP, INPUT_SIZE, dim=True)
        
        feed_dict = {
                model.x  : mix,
                model.y1 : t1,
                model.y2 : t2,
                model.keep_prob : 1.0,
                model.training : False
            }
        if idx==(DATA_SIZE/(BATCH_SIZE*TIME_STEP))-1:
            model.sequence_length = sequence_length[sp_list[speaker]]
            # stochastic gradient descent
            cost, pred1, pred2 = sess.run(
                [model.cost, model.pred1, model.pred2],
                feed_dict
                )
            model.init_state_assign()
            model.sequence_length = [TIME_STEP for i in xrange(0, BATCH_SIZE)]
        else:
            # stochastic gradient descent
            cost, pred1, pred2 = sess.run(
                [model.cost, model.pred1, model.pred2],
                feed_dict
                )
            model.init_state_assign()
        
        total_pred1 = np.append(total_pred1, pred1, axis=0)
        total_pred2 = np.append(total_pred2, pred2, axis=0)
        # cost
        total_cost = total_cost + cost
            
total_cost = total_cost/(TOTAL_SIZE)
print 'The cost of model: ', total_cost
MSE = np.concatenate((MSE, [total_cost]), axis=0)
total_pred1 = total_pred1[1:, :]
total_pred2 = total_pred2[1:, :]
total_pred1 = np.transpose(total_pred1)
total_pred2 = np.transpose(total_pred2)
            
print('----------End----------')

In [ ]:
# np.savetxt('../Result/'+data+'/SNR='+str(SNR)+'/Test1/LSTM'+'_d='+str(HIDDEN_SIZE)+"_sp_"+str(NUM_SPEAKER)+'_prediction1.csv', total_pred1, delimiter=",")
# np.savetxt('../Result/'+data+'/SNR='+str(SNR)+'/Test1/LSTM'+'_d='+str(HIDDEN_SIZE)+"_sp_"+str(NUM_SPEAKER)+'_prediction2.csv', total_pred1, delimiter=",")
np.savetxt('../Result/'+data+'/SNR='+str(SNR)+'/Test1/MRNN'+'_d='+str(HIDDEN_SIZE)+'_K='+str(K)+"_sp_"+str(NUM_SPEAKER)+'_prediction1.csv', total_pred1, delimiter=",")
np.savetxt('../Result/'+data+'/SNR='+str(SNR)+'/Test1/MRNN'+'_d='+str(HIDDEN_SIZE)+'_K='+str(K)+"_sp_"+str(NUM_SPEAKER)+'_prediction2.csv', total_pred1, delimiter=",")

# Unseen speaker test

In [ ]:
NUM_SPEAKER_TEST = 6
mix_test2, t1_test2, t2_test2, order, sequence_length = get_data_test(URL+'Test2/Mix/', URL+'Test2/Target1/', URL+'Test2/Target2/', 
                                                                      NUM_SPEAKER_TEST, BATCH_SIZE, TIME_STEP)

In [ ]:
total_cost  = 0
TOTAL_SIZE  = 0
total_pred1 = np.zeros((1, 513))
total_pred2 = np.zeros((1, 513))
sp_list     = mix_test2.keys()
sp_list.sort()
for speaker in range(NUM_SPEAKER_TEST):
    BATCH_START = 0 # to record starting point of batch we want to extract
    DATA_SIZE   = mix_test2[sp_list[speaker]].shape[0]
    TOTAL_SIZE  = TOTAL_SIZE + DATA_SIZE
    for idx in range((DATA_SIZE/(BATCH_SIZE*TIME_STEP))):
        BATCH_START = idx*BATCH_SIZE*TIME_STEP
        mix, t1, t2 = get_batch_test(
            mix_test2[sp_list[speaker]], t1_test2[sp_list[speaker]], t2_test2[sp_list[speaker]], 
            BATCH_START, BATCH_SIZE, TIME_STEP, INPUT_SIZE, dim=True)
        
        feed_dict = {
                model.x  : mix,
                model.y1 : t1,
                model.y2 : t2,
                model.keep_prob : 1.0,
                model.training : False
            }
        
        if idx==(DATA_SIZE/(BATCH_SIZE*TIME_STEP))-1:
            model.sequence_length = sequence_length[sp_list[speaker]]
            # stochastic gradient descent
            cost, pred1, pred2 = sess.run(
                [model.cost, model.pred1, model.pred2],
                feed_dict
                )
            
            model.init_state_assign()
            model.sequence_length = [TIME_STEP for i in xrange(0, BATCH_SIZE)]
        else:
            # stochastic gradient descent
            cost, pred1, pred2 = sess.run(
                [model.cost, model.pred1, model.pred2],
                feed_dict
                )
            model.init_state_assign()
        
        total_pred1 = np.append(total_pred1, pred1, axis=0)
        total_pred2 = np.append(total_pred2, pred2, axis=0)
        # cost
        total_cost = total_cost + cost
            
total_cost = total_cost/(TOTAL_SIZE)
print 'The cost of model: ', total_cost
MSE = np.concatenate((MSE, [total_cost]), axis=0)
total_pred1 = total_pred1[1:, :]
total_pred2 = total_pred2[1:, :]
total_pred1 = np.transpose(total_pred1)
total_pred2 = np.transpose(total_pred2)
            
print('----------End----------')

In [ ]:
# np.savetxt('../Result/'+data+'/SNR='+str(SNR)+'/Test2/LSTM'+'_d='+str(HIDDEN_SIZE)+"_sp_"+str(NUM_SPEAKER)+'_prediction1.csv', total_pred1, delimiter=",")
# np.savetxt('../Result/'+data+'/SNR='+str(SNR)+'/Test2/LSTM'+'_d='+str(HIDDEN_SIZE)+"_sp_"+str(NUM_SPEAKER)+'_prediction2.csv', total_pred1, delimiter=",")
np.savetxt('../Result/'+data+'/SNR='+str(SNR)+'/Test2/MRNN'+'_d='+str(HIDDEN_SIZE)+'_K='+str(K)+"_sp_"+str(NUM_SPEAKER)+'_prediction1.csv', total_pred1, delimiter=",")
np.savetxt('../Result/'+data+'/SNR='+str(SNR)+'/Test2/MRNN'+'_d='+str(HIDDEN_SIZE)+'_K='+str(K)+"_sp_"+str(NUM_SPEAKER)+'_prediction2.csv', total_pred1, delimiter=",")